In [ ]:
TD3 only used for Continuous Action space

In [1]:
import pybullet_envs
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from gym import wrappers
from torch.autograd import Variable
from collections import deque

In [7]:
# Step 1
class ReplayBuffer(object):
    def __init__(self, max_size = 1e6):
        self.storage = []
        self.max_size = max_size
        self.ptr = 0 #pointer to different cells in the Queue
    
    def add(self, transition):
        if len(self.storage) == self.max_size:
            self.storage[int(self.ptr)] = transition
            self.ptr = (self.ptr + 1)%self.max_size
        else:
            self.storage.append(transition)
    
    def sample(self, batch_size):
        ind = np.random.randint(0, len(self.storage), size = batch_size)
        batch_states, batch_next_states, batch_actions, batch_rewards, batch_done = [],[],[],[],[]
        for i in ind:
            state, next_state, action, reward, done = self.storage[i]
            batch_states.append(np.array(state, copy=False))
            batch_next_states.append(np.array(state, copy=False))
            batch_actions.append(np.array(action, copy=False))
            batch_rewards.append(np.array(reward, copy=False))
            batch_done.append(np.array(done, copy=False))
        return np.array(batch_states), np.array(batch_next_states), np.array(batch_actions), np.array(batch_rewards).reshape(-1,1), np.array(batch_done).reshape(-1,1)
          


In [8]:
# Step 2
# inheriting tools from pytorch nn module
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, max_action):
        super(Actor, self).__init__()
        self.layer1 = nn.Linear(state_dim, 400)
        self.layer2 = nn.Linear(400, 300)
        self.layer3 = nn.Linear(300, action_dim)
        self.max_action = max_action
    
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.max_action*torch.tanh(self.layer3(x))

        return x
        

In [9]:
# Step 3
class Critic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()
        # Defining the first Critic NN
        self.layer1 = nn.Linear(state_dim + action_dim, 400)
        self.layer2 = nn.Linear(400, 300)
        self.layer3 = nn.Linear(300, 1)

        # Defining the second Critic NN
        self.layer4 = nn.Linear(state_dim + action_dim, 400)
        self.layer5 = nn.Linear(400, 300)
        self.layer6 = nn.Linear(300, 1)
    
    def forward(self, x, u):
        # State x and action u
        xu = torch.cat([x, u], axis = 1)
        xu1 = F.relu(self.layer1(xu))
        xu1 = F.relu(self.layer2(xu1))
        xu1 = self.layer3(xu1)

        xu2 = F.relu(self.layer4(xu))
        xu2 = F.relu(self.layer5(xu2))
        xu2 = self.layer6(xu2)

        return xu1,xu2
    
    def Q1_forward(self, x, u):
        # State x and action u
        xu = torch.cat([x, u], axis = 1)
        xu1 = F.relu(self.layer1(xu))
        xu1 = F.relu(self.layer2(xu1))
        xu1 = self.layer3(xu1)

        return xu1

In [23]:
# Step 4-15
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

class TD3(object):
    def __init__(self, state_dim, action_dim, max_action):
        self.actor = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters())

        self.critic = Critic(state_dim, action_dim).to(device)
        self.critic_target = Critic(state_dim, action_dim).to(device)
        self.critic_target.load_state_dict(self.critic.state_dict())
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters())

        self.max_action = max_action
    
    def select_action(self, state):
        state = torch.Tensor(state.reshape(1,-1)).to(device)
        # Numpy -> Torch Tensor -> Numpy last step becoz, adding noise is easier in numpy
        return self.actor(state).gpu().data.numpy().flatten()
    # gamma->discount factor, tau->poliyak avg
    def train(self, replay_buffer, num_iter, batch_size=100, gamma=0.99, tau=0.005, policy_noise=0.2, noise_clip = 0.5, policy_freq=2):

        for it in range(num_iter):

            # Step-4
            batch_states,batch_next_states,batch_actions,batch_rewards,batch_done = replay_buffer.sample(batch_size)
            state = torch.Tensor(batch_states).to(device)
            next_state = torch.Tensor(batch_next_states).to(device)
            action = torch.Tensor(batch_actions).to(device) 
            reward = torch.Tensor(batch_rewards).to(device)
            done = torch.Tensor(batch_done).to(device)

            # Step-5
            next_action = self.actor_target.forward(next_state)

            # Step-6 adding noise
            # Creating noise of normal_distribution
            noise = torch.Tensor(batch_actions).data.normal_(0, policy_noise).to(device)
            noise = noise.clamp(-noise_clip, noise_clip)
            next_action = (next_action + noise).clamp(-self.max_action, self.max_action)

            # Step-7
            target_Q1, target_Q2 = self.critic_target.forward(next_state, next_action)

            # Step-8
            target_Q = torch.min(target_Q1,target_Q2)

            # step-9
            # target_Q = reward + discount*target_Q*(1 - done)
            # For faster computation:
            target_Q = reward + (discount*target_Q*(1 - done)).detach()

            # Step-10
            current_Q1, current_Q2 = self.critic.forward(state,action)

            # Step-11
            # Loss
            critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)
            
            # Step-12
            # Backprop ---------------- zero_grad->initializes all gradients to zero
            self.critic_optimizer.zero_grad()
            critic_loss.backward()
            self.critic_optimizer.step()

            # Step-13
            # Gradient Ascent hence we do the opposite of SGD as we want to maximise the Q value.
            if it % policy_freq == 0:
                actor_loss = -self.critic.Q1_forward(state, self.actor.forward(state)).mean() #all the calculations are still done on a batch. So, we take the mean.
                self.actor_optimizer.zero_grad()
                actor_loss.backward()
                self.actor_optimizer.step()

                # Step-14,15
                # Polyak averaging
                for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
                    target_param.data.copy_(tau*param.data +(1-tau)*target_param.data)
                
                for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
                    target_param.data.copy_(tau*param.data +(1-tau)*target_param.data)
    def save(self, filename, directory):
        torch.save(self.actor.state_dict(), '%s/%s_actor.pth'%(directory,filename))
        torch.save(self.critic.state_dict(), '%s/%s_critic.pth'%(directory,filename))
    
    def load(self, filename, directory):
        self.actor.load_state_dict(torch.load(self.actor.state_dict(), '%s/%s_actor.pth'%(directory,filename)))
        self.critic.load_state_dict(torch.load(self.critic.state_dict(), '%s/%s_critic.pth'%(directory,filename)))
        

cuda


In [11]:
def evaluate_policy(policy, eval_episodes=10):
  avg_reward = 0.
  for _ in range(eval_episodes):
    obs = env.reset()
    done = False
    while not done:
      action = policy.select_action(np.array(obs))
      obs, reward, done, _ = env.step(action)
      avg_reward += reward
  avg_reward /= eval_episodes
  print ("---------------------------------------")
  print ("Average Reward over the Evaluation Step: %f" % (avg_reward))
  print ("---------------------------------------")
  return avg_reward

In [12]:
env_name = "AntBulletEnv-v0" # Name of a environment (set it to any Continous environment you want)
seed = 0 # Random seed number
start_timesteps = 1e4 # Number of iterations/timesteps before which the model randomly chooses an action, and after which it starts to use the policy network
eval_freq = 5e3 # How often the evaluation step is performed (after how many timesteps)
max_timesteps = 5e5 # Total number of iterations/timesteps
save_models = True # Boolean checker whether or not to save the pre-trained model
expl_noise = 0.1 # Exploration noise - STD value of exploration Gaussian noise
batch_size = 100 # Size of the batch
discount = 0.99 # Discount factor gamma, used in the calculation of the total discounted reward
tau = 0.005 # Target network update rate
policy_noise = 0.2 # STD of Gaussian noise added to the actions for the exploration purposes
noise_clip = 0.5 # Maximum value of the Gaussian noise added to the actions (policy)
policy_freq = 2 # Number of iterations to wait before the policy network (Actor model) is updated

In [13]:
file_name = "%s_%s_%s" % ("TD3", env_name, str(seed))
print ("---------------------------------------")
print ("Settings: %s" % (file_name))
print ("---------------------------------------")

---------------------------------------
Settings: TD3_AntBulletEnv-v0_0
---------------------------------------


In [14]:
if not os.path.exists("./results"):
  os.makedirs("./results")
if save_models and not os.path.exists("./pytorch_models"):
  os.makedirs("./pytorch_models")

In [15]:
env = gym.make(env_name)

In [16]:
env.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])

In [19]:
policy = TD3(state_dim, action_dim, max_action)
replay_buffer = ReplayBuffer()
evaluations = [evaluate_policy(policy)]

---------------------------------------
Average Reward over the Evaluation Step: 8.057792
---------------------------------------


In [20]:
def mkdir(base, name):
    path = os.path.join(base, name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path
work_dir = mkdir('exp', 'brs')
monitor_dir = mkdir(work_dir, 'monitor')
max_episode_steps = env._max_episode_steps
save_env_vid = False
if save_env_vid:
  env = wrappers.Monitor(env, monitor_dir, force = True)
  env.reset()

In [21]:
total_timesteps = 0
timesteps_since_eval = 0
episode_num = 0
done = True
t0 = time.time()

In [24]:
# We start the main loop over 500,000 timesteps
while total_timesteps < max_timesteps:
  
  # If the episode is done
  if done:

    # If we are not at the very beginning, we start the training process of the model
    if total_timesteps != 0:
      print("Total Timesteps: {} Episode Num: {} Reward: {}".format(total_timesteps, episode_num, episode_reward))
      policy.train(replay_buffer, episode_timesteps, batch_size, discount, tau, policy_noise, noise_clip, policy_freq)

    # We evaluate the episode and we save the policy
    if timesteps_since_eval >= eval_freq:
      timesteps_since_eval %= eval_freq
      evaluations.append(evaluate_policy(policy))
      policy.save(file_name, directory="./pytorch_models")
      np.save("./results/%s" % (file_name), evaluations)
    
    # When the training step is done, we reset the state of the environment
    obs = env.reset()
    
    # Set the Done to False
    done = False
    
    # Set rewards and episode timesteps to zero
    episode_reward = 0
    episode_timesteps = 0
    episode_num += 1
  
  # Before 10000 timesteps, we play random actions
  if total_timesteps < start_timesteps:
    action = env.action_space.sample()
  else: # After 10000 timesteps, we switch to the model
    action = policy.select_action(np.array(obs))
    # If the explore_noise parameter is not 0, we add noise to the action and we clip it
    if expl_noise != 0:
      action = (action + np.random.normal(0, expl_noise, size=env.action_space.shape[0])).clip(env.action_space.low, env.action_space.high)
  
  # The agent performs the action in the environment, then reaches the next state and receives the reward
  new_obs, reward, done, _ = env.step(action)
  
  # We check if the episode is done
  done_bool = 0 if episode_timesteps + 1 == env._max_episode_steps else float(done)
  
  # We increase the total reward
  episode_reward += reward
  
  # We store the new transition into the Experience Replay memory (ReplayBuffer)
  replay_buffer.add((obs, new_obs, action, reward, done_bool))

  # We update the state, the episode timestep, the total timesteps, and the timesteps since the evaluation of the policy
  obs = new_obs
  episode_timesteps += 1
  total_timesteps += 1
  timesteps_since_eval += 1

# We add the last policy evaluation to our list of evaluations and we save our model
evaluations.append(evaluate_policy(policy))
if save_models: policy.save("%s" % (file_name), directory="./pytorch_models")
np.save("./results/%s" % (file_name), evaluations)

Total Timesteps: 33221 Episode Num: 46 Reward: 422.8485920488165
Total Timesteps: 34221 Episode Num: 47 Reward: 441.7061372441606
Total Timesteps: 35221 Episode Num: 48 Reward: 420.3218206931139
---------------------------------------
Average Reward over the Evaluation Step: 559.818325
---------------------------------------
Total Timesteps: 36221 Episode Num: 49 Reward: 590.324727722915
Total Timesteps: 37221 Episode Num: 50 Reward: 427.5155814534249
Total Timesteps: 38221 Episode Num: 51 Reward: 385.5154933630216
Total Timesteps: 39221 Episode Num: 52 Reward: 460.70406288682244
Total Timesteps: 39241 Episode Num: 53 Reward: 4.776833908606749
Total Timesteps: 39261 Episode Num: 54 Reward: 5.884747882192498
Total Timesteps: 39883 Episode Num: 55 Reward: 338.2725627999538
Total Timesteps: 39916 Episode Num: 56 Reward: 8.125204243082996
Total Timesteps: 39936 Episode Num: 57 Reward: 3.544224400309174
Total Timesteps: 40936 Episode Num: 58 Reward: 631.3940637496123
-----------------------

KeyboardInterrupt: 

In [26]:
def evaluate_policy(policy, eval_episodes=10):
  avg_reward = 0.
  for _ in range(eval_episodes):
    obs = env.reset()
    done = False
    while not done:
      action = policy.select_action(np.array(obs))
      obs, reward, done, _ = env.step(action)
      avg_reward += reward
  avg_reward /= eval_episodes
  print ("---------------------------------------")
  print ("Average Reward over the Evaluation Step: %f" % (avg_reward))
  print ("---------------------------------------")
  return avg_reward

env_name = "AntBulletEnv-v0"
seed = 0

file_name = "%s_%s_%s" % ("TD3", env_name, str(seed))
print ("---------------------------------------")
print ("Settings: %s" % (file_name))
print ("---------------------------------------")

eval_episodes = 10
save_env_vid = True
env = gym.make(env_name)
max_episode_steps = env._max_episode_steps
if save_env_vid:
  env = wrappers.Monitor(env, monitor_dir, force = True)
  env.reset()
env.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])
policy = TD3(state_dim, action_dim, max_action)
policy.load(file_name, './pytorch_models/')
_ = evaluate_policy(policy, eval_episodes=eval_episodes)

---------------------------------------
Settings: TD3_AntBulletEnv-v0_0
---------------------------------------


DependencyNotInstalled: Found neither the ffmpeg nor avconv executables. On OS X, you can install ffmpeg via `brew install ffmpeg`. On most Ubuntu variants, `sudo apt-get install ffmpeg` should do it. On Ubuntu 14.04, however, you'll need to install avconv with `sudo apt-get install libav-tools`.